<a href="https://colab.research.google.com/github/AnuragSingh7292/Remote-sensing-image-retrieval-deep-learning-/blob/main/MAP%26ANMRR_of_ALL_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# After running, follow the authentication steps


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


MAP

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load extracted features and labels for VGG16
# save_dir = "/content/drive/MyDrive/NewONeFeature_IFK_Vectors"
# save_dir = "/content/drive/MyDrive/NewONeFeature_VGG_Vectors"
# save_dir = "/content/drive/MyDrive/NewONeFeature_PCA_PrePro_Vectors"
# save_dir = "/content/drive/MyDrive/Feature_Vectors_mobileNet_patternNet"
save_dir = "/content/drive/MyDrive/inception_Feature_Vectors_patternNet"



# save_dir = "/content/drive/MyDrive/resnet_features_patternNet_vectors"

features = np.load(f"{save_dir}/inceptionv3_fc6_32_ucmerced_features.npy")
labels = np.load(f"{save_dir}/labelsucmerced.npy")

# Compute Mean Average Precision (MAP)
def compute_map(features, labels):
    num_images = features.shape[0]
    similarity_matrix = cosine_similarity(features)  # Compute cosine similarity
    average_precisions = []

    for i in range(num_images):
        query_label = labels[i]
        similarities = similarity_matrix[i]

        # Sort images by similarity (descending order) and exclude self-match
        sorted_indices = np.argsort(-similarities)
        sorted_indices = sorted_indices[sorted_indices != i]  # Remove self-
        # sorted_indices = np.delete(sorted_indices, np.where(sorted_indices == i))  # More robust
        sorted_labels = labels[sorted_indices]

        # Compute relevance scores (1 if label matches, else 0)
        relevant_items = (sorted_labels == query_label).astype(int)
        num_relevant = np.sum(relevant_items)

        # If no relevant items exist, AP is 0
        if num_relevant == 0:
            average_precisions.append(0)
            continue

        # Compute precision at each rank
        cumulative_relevance = np.cumsum(relevant_items)
        precision_at_k = cumulative_relevance / (np.arange(len(relevant_items)) + 1)

        # Compute Average Precision (AP)
        average_precision = np.sum(precision_at_k * relevant_items) / num_relevant
        average_precisions.append(average_precision)

    # Compute Mean Average Precision (MAP)
    mean_average_precision = np.mean(average_precisions)
    return mean_average_precision

# Compute and print MAP for VGG16
map_score = compute_map(features, labels)
print(f"✅ MAP for combine ResNet 50&101 : {map_score * 100:.2f}%")  # Convert to percentage


✅ MAP for combine ResNet 50&101 : 56.62%


ANMRR

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

def compute_anmrr(labels, distance_matrix):
    num_images = len(labels)
    anmrr_values = []

    for i in range(num_images):
        # Sort indices based on distance (ascending order)
        sorted_indices = np.argsort(distance_matrix[i])
        sorted_indices = sorted_indices[sorted_indices != i]  # Exclude self-match

        # Retrieve the sorted labels
        sorted_labels = labels[sorted_indices]

        # Identify relevant items
        relevant_items = (sorted_labels == labels[i])
        num_relevant = np.sum(relevant_items)

        if num_relevant == 0:
            anmrr_values.append(1)  # Worst case if no relevant images found
            continue

        # Compute rank positions of relevant items
        rank_positions = np.where(relevant_items)[0] + 1  # Convert to 1-based indexing

        # Compute Modified Retrieval Rank (MRR)
        mrr = np.sum(rank_positions) / num_relevant

        # Compute Normalized Modified Retrieval Rank (NMRR)
        # K = min(2 * num_relevant, num_images)  # Dynamic cut-off based on relevant items
        K = num_images  # Since we are considering all images
        nmrr = (mrr - 0.5 * (1 + num_relevant)) / (1.25 * K - 0.5 * (1 + num_relevant))

        anmrr_values.append(nmrr)

    # Mean of NMRR values
    anmrr = np.mean(anmrr_values)
    return anmrr


# Compute and print ANMRR for each model (all images)

features = np.load(f"/content/drive/MyDrive/inception_Feature_Vectors_patternNet/inceptionv3_fc6_32_ucmerced_features.npy")
labels = np.load(f"/content/drive/MyDrive/inception_Feature_Vectors_patternNet/labelsucmerced.npy")

# Compute distance matrix efficiently using cdist (Euclidean distance)
distance_matrix = cdist(features, features, metric='euclidean')

# Compute ANMRR for all images
anmrr_score = compute_anmrr(labels, distance_matrix)
print(f"ANMRR for combine  ResNet 50&101 : {anmrr_score:.4f}")


ANMRR for combine  ResNet 50&101 : 0.0778
